In [1]:
# pip install pyserial ipywidgets plotly

import threading, time
import numpy as np
import serial, serial.tools.list_ports
import ipywidgets as W
import plotly.graph_objects as go
from IPython.display import display


In [3]:

HEADER = "# BMM350 CSV"

# --- UI ---
ports = [p.device for p in serial.tools.list_ports.comports()] or ["COM8"]
port   = W.Dropdown(options=ports, description="Port")
baud   = W.Dropdown(options=[115200,230400,460800,921600], value=115200, description="Baud")
pts    = W.IntText(value=1000, description="Points")
start  = W.Button(description="▶ Start", button_style="success")
stop   = W.Button(description="■ Stop", button_style="danger", disabled=True)
status = W.HTML("<b>Idle.</b>")
refresh= W.Button(description="Refresh Ports")
ui = W.HBox([port, baud, pts, start, stop, refresh]); display(ui)

fig = go.FigureWidget(layout=dict(template="plotly_white", height=420,
                                  xaxis_title="Sample", yaxis_title="Value"))
for name in ("X","Y","Z"): fig.add_scatter(name=name, mode="lines")
display(fig); display(status)

# --- State ---
stop_flag = threading.Event()
xs, ys, zs = [], [], []

def set_status(msg): status.value = msg

def on_refresh(_):
    opts = [p.device for p in serial.tools.list_ports.comports()] or ["COM8"]
    cur = port.value
    port.options = opts
    
    if cur in opts: port.value = cur
refresh.on_click(on_refresh)

def worker(port_name, baudrate, maxpts, batch=20, timeout=1.0, header_timeout=8.0):
    try:
        ser = serial.Serial(port_name, baudrate, timeout=timeout)
    except Exception as e:
        set_status(f"<b>Open failed:</b> {e}"); start.disabled=False; stop.disabled=True; return
    try:
        # header
        t0 = time.time()
        while time.time()-t0 < header_timeout and not stop_flag.is_set():
            if ser.readline().decode(errors="ignore").strip().startswith(HEADER): break
        else:
            set_status("<b>HEADER not seen.</b>"); return
        # read
        xs.clear(); ys.clear(); zs.clear()
        set_status(f"Collecting… 0/{maxpts}")
        emitted = 0
        while len(xs) < maxpts and not stop_flag.is_set():
            s = ser.readline().decode(errors="ignore").strip()
            if not s or s.startswith("#") or s=="DONE": continue
            parts = s.split(",")
            if len(parts)!=3: continue
            try: x,y,z = map(float, parts)
            except: continue
            xs.append(x); ys.append(y); zs.append(z)
            if len(xs)-emitted >= batch or len(xs)==maxpts:
                idx = np.arange(len(xs))
                fig.data[0].x, fig.data[0].y = idx, xs
                fig.data[1].x, fig.data[1].y = idx, ys
                fig.data[2].x, fig.data[2].y = idx, zs
                fig.update_xaxes(range=[0, max(len(xs),100)])
                set_status(f"Collecting… {len(xs)}/{maxpts}")
                emitted = len(xs)
        set_status(f"<b>Done.</b> {len(xs)} points.")
    except Exception as e:
        set_status(f"<b>Error:</b> {e}")
    finally:
        try: ser.close()
        except: pass
        stop.disabled=True; start.disabled=False; stop_flag.clear()

def on_start(_):
    if start.disabled: return
    start.disabled=True; stop.disabled=False; stop_flag.clear()
    set_status(f"Opening {port.value} @ {baud.value}…")
    threading.Thread(target=worker, args=(port.value,int(baud.value),int(pts.value)), daemon=True).start()
start.on_click(on_start)

def on_stop(_):
    stop_flag.set(); stop.disabled=True; set_status("Stopping…")
stop.on_click(on_stop)


FigureWidget({
    'data': [{'mode': 'lines', 'name': 'X', 'type': 'scatter', 'uid': 'b123e900-de48-4c1f-ae97-ac746be9804f'},
             {'mode': 'lines', 'name': 'Y', 'type': 'scatter', 'uid': 'd1b033ae-0e3a-4fad-aee7-cfa29c352041'},
             {'mode': 'lines', 'name': 'Z', 'type': 'scatter', 'uid': 'e980edcd-a396-4c06-84f5-2b60c4dcc6ce'}],
    'layout': {'height': 420,
               'template': '...',
               'xaxis': {'title': {'text': 'Sample'}},
               'yaxis': {'title': {'text': 'Value'}}}
})

HTML(value='<b>Idle.</b>')